In [1]:
using LightGraphs, RCall, LightGraphsFlows, Suppressor, Gadfly, DataFrames, CSV, PyCall, Compose

In [2]:
R"library('genlasso')";
include("code/BGSM_general.jl");
include("code/BGSM_biclust.jl");
include("code/model_selection.jl");
include("code/l0pen.jl");
using PyCall
@pyimport GraphSegment
@pyimport numpy

Loading required package: Matrix
Loading required package: igraph

Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union



In [15]:
function BGSM(y, D; mult = 1.1, num = 30, st = 1e-3 * 5)
    v0_range = st * mult.^(0:num-1); a = Inf; bgsm1 = 0; bgsm2 = 0;
    t = 0;
    for i = 1:length(v0_range)
        tic();
        temp = BGSM_general(y,D, v0 = v0_range[i], v1 = 1e4, A = n, b = n,
                                    convtol = 1e-8, iter = 100, verbose = false);
        t = t + toq();
        bgsm_ms = model_selection(temp);
        #println(sum(D * bgsm_ms[:beta] .> 1e-8))
        #println(bgsm_ms[:score])
        if bgsm_ms[:score] < a
            a = bgsm_ms[:score];
            bgsm1 = temp[:alpha] + temp[:theta];
            bgsm2 = bgsm_ms[:t_full];
        end
    end
    
    return Dict([(:thetahat, bgsm1), (:theta, bgsm2), (:t, t)])
end

function genlasso(y)
    t = 0;
    tic();
    @rput y;
    R"out =  trendfilter(y, ord=0);";
    t = t + toq();
    @suppress begin
    R"cv = cv.trendfilter(out);
    out_fl = coef(out, lambda=cv$lambda.min, verbose = FALSE)$beta;";
    end;
    @rget out_fl;
    
    return Dict([(:theta, out_fl), (:t, t)])
end

function l0pen(y; B = 10, mult = 2.0, sigma = 1)
    t = 0;
    edges = GraphSegment.Edges_1D(n);
    err   = zeros(10, B);
    out_l0pen = 0;
    tic();
    for b = 1:B
        for i = 1:10
            @suppress begin
            out_l0pen = GraphSegment.GraphSegment(y,edges, mult^(i-4), delta=.1);
            #sigmahat  = norm(y - out_l0pen)/sqrt(n);
            alpha     = 0.2;
            z         = alpha * sigma * randn(n);
            y1        = y + z;
            y2        = y - z/alpha^2;
            mu1       = GraphSegment.GraphSegment(y1,edges, mult^(i-4), delta=.1);
            err[i,b]  = norm(y2 - mu1)^2/n;
            end;
        end
    end
    t = toq();
    
    score = sum(err, 2);
    minind = findmin(score)[2]
    @suppress begin
    out_l0pen = GraphSegment.GraphSegment(y,edges, mult^(minind - 4), delta=.1);
    end;
    
    return Dict([(:theta, out_l0pen), (:err, err), (:t, t)])
end

l0pen (generic function with 1 method)

In [16]:
srand(1);
delta = repeat([1:5;4;3;4;2;3], inner = 2) .* (-1).^(1:20); randn();
theta = cumsum(delta);
theta = repeat(theta, inner = 50);
y     = theta + .5 * randn(1000);
n     = length(y);
g     = PathGraph(n);
D     = -incidence_matrix(g, oriented = true)';

In [19]:
tind = find(abs.(D * theta) .> 1e-8);

In [37]:
mse_bgsm   = zeros(10); mse_genlasso = zeros(10); mse_l0pen = zeros(10);
theta_bgsm = zeros(1000,10); theta_genlasso = zeros(1000,10); theta_l0pen = zeros(1000,10);
fdr        = zeros(3,10); power = zeros(3,10);
t          = zeros(3,10);
fit1 = zeros(10,5); fit2 = zeros(10,5); fit3 = zeros(10,5);
fitfdr = zeros(3,5); fitpower = zeros(3,5); fitt = zeros(3,5);

srand(2018);
for i = 1:10
    y     = theta + .5 * randn(1000);
    tic();
    out_genlasso = genlasso(y);
    t[2,i] = toq();
    print("genlasso done; ")
    tic();
    out_bgsm     = BGSM(y, D, mult = 1.3, num = 20, st = 1e-3 * 2);
    t[1,i] = toq();
    print("bgsm done; ")
    tic();
    out_l0pen    = l0pen(y; B = 10, mult = 2.0);
    t[3,i] = toq();
    print("l0pen done; \n")
    nzind = find(abs.(D * out_bgsm[:theta]) .> 1e-8);
    fdr[1,i]      = 1 - length(findin(tind, nzind))/length(nzind);
    power[1,i]    = length(findin(tind, nzind))/length(tind)
    nzind = find(abs.(D * out_genlasso[:theta]) .> 1e-8);
    fdr[2,i]      = 1 - length(findin(tind, nzind))/length(nzind);
    power[2,i]    = length(findin(tind, nzind))/length(tind)
    nzind = find(abs.(D * out_l0pen[:theta]) .> 1e-8);
    fdr[3,i]      = 1 - length(findin(tind, nzind))/length(nzind);
    power[3,i]    = length(findin(tind, nzind))/length(tind)
        
    mse_bgsm[i] = norm(theta - out_bgsm[:theta])^2/n;
    mse_genlasso[i] = norm(theta - out_genlasso[:theta])^2/n;
    mse_l0pen[i] = norm(theta - out_l0pen[:theta])^2/n;
    
    theta_bgsm[:,i] = out_bgsm[:theta];
    theta_genlasso[:,i] = out_genlasso[:theta];
    theta_l0pen[:,i] = out_l0pen[:theta];
end
fit1[:,5] = copy(mse_bgsm);
fit2[:,5] = copy(mse_genlasso);
fit3[:,5] = copy(mse_l0pen);
fitfdr[:,5] = mean(fdr, 2)[:];
fitpower[:,5] = mean(power, 2)[:];
fitt[:,5] = mean(t, 2)[:];



srand(2018);
for i = 1:10
    y     = theta + .4 * randn(1000);
    tic();
    out_genlasso = genlasso(y);
    t[2,i] = toq();
    print("genlasso done; ")
    tic();
    out_bgsm     = BGSM(y, D, mult = 1.3, num = 20, st = 1e-3 * 2);
    t[1,i] = toq();
    print("bgsm done; ")
    tic();
    out_l0pen    = l0pen(y; B = 10, mult = 2.0);
    t[3,i] = toq();
    print("l0pen done; \n")
    nzind = find(abs.(D * out_bgsm[:theta]) .> 1e-8);
    fdr[1,i]      = 1 - length(findin(tind, nzind))/length(nzind);
    power[1,i]    = length(findin(tind, nzind))/length(tind)
    nzind = find(abs.(D * out_genlasso[:theta]) .> 1e-8);
    fdr[2,i]      = 1 - length(findin(tind, nzind))/length(nzind);
    power[2,i]    = length(findin(tind, nzind))/length(tind)
    nzind = find(abs.(D * out_l0pen[:theta]) .> 1e-8);
    fdr[3,i]      = 1 - length(findin(tind, nzind))/length(nzind);
    power[3,i]    = length(findin(tind, nzind))/length(tind)
        
    mse_bgsm[i] = norm(theta - out_bgsm[:theta])^2/n;
    mse_genlasso[i] = norm(theta - out_genlasso[:theta])^2/n;
    mse_l0pen[i] = norm(theta - out_l0pen[:theta])^2/n;
    
    theta_bgsm[:,i] = out_bgsm[:theta];
    theta_genlasso[:,i] = out_genlasso[:theta];
    theta_l0pen[:,i] = out_l0pen[:theta];
end
fit1[:,4] = copy(mse_bgsm);
fit2[:,4] = copy(mse_genlasso);
fit3[:,4] = copy(mse_l0pen);
fitfdr[:,4] = mean(fdr, 2)[:];
fitpower[:,4] = mean(power, 2)[:];
fitt[:,4] = mean(t, 2)[:];



srand(2018);
for i = 1:10
    y     = theta + .3 * randn(1000);
    tic();
    out_genlasso = genlasso(y);
    t[2,i] = toq();
    print("genlasso done; ")
    tic();
    out_bgsm     = BGSM(y, D, mult = 1.3, num = 20, st = 1e-3 * 2);
    t[1,i] = toq();
    print("bgsm done; ")
    tic();
    out_l0pen    = l0pen(y; B = 10, mult = 2.0);
    t[3,i] = toq();
    print("l0pen done; \n")
    nzind = find(abs.(D * out_bgsm[:theta]) .> 1e-8);
    fdr[1,i]      = 1 - length(findin(tind, nzind))/length(nzind);
    power[1,i]    = length(findin(tind, nzind))/length(tind)
    nzind = find(abs.(D * out_genlasso[:theta]) .> 1e-8);
    fdr[2,i]      = 1 - length(findin(tind, nzind))/length(nzind);
    power[2,i]    = length(findin(tind, nzind))/length(tind)
    nzind = find(abs.(D * out_l0pen[:theta]) .> 1e-8);
    fdr[3,i]      = 1 - length(findin(tind, nzind))/length(nzind);
    power[3,i]    = length(findin(tind, nzind))/length(tind)
        
    mse_bgsm[i] = norm(theta - out_bgsm[:theta])^2/n;
    mse_genlasso[i] = norm(theta - out_genlasso[:theta])^2/n;
    mse_l0pen[i] = norm(theta - out_l0pen[:theta])^2/n;
    
    theta_bgsm[:,i] = out_bgsm[:theta];
    theta_genlasso[:,i] = out_genlasso[:theta];
    theta_l0pen[:,i] = out_l0pen[:theta];
end
fit1[:,3] = copy(mse_bgsm);
fit2[:,3] = copy(mse_genlasso);
fit3[:,3] = copy(mse_l0pen);
fitfdr[:,3] = mean(fdr, 2)[:];
fitpower[:,3] = mean(power, 2)[:];
fitt[:,3] = mean(t, 2)[:];




srand(2018);
for i = 1:10
    y     = theta + .2 * randn(1000);
    tic();
    out_genlasso = genlasso(y);
    t[2,i] = toq();
    print("genlasso done; ")
    tic();
    out_bgsm     = BGSM(y, D, mult = 1.3, num = 20, st = 1e-3 * 2);
    t[1,i] = toq();
    print("bgsm done; ")
    tic();
    out_l0pen    = l0pen(y; B = 10, mult = 2.0);
    t[3,i] = toq();
    print("l0pen done; \n")
    nzind = find(abs.(D * out_bgsm[:theta]) .> 1e-8);
    fdr[1,i]      = 1 - length(findin(tind, nzind))/length(nzind);
    power[1,i]    = length(findin(tind, nzind))/length(tind)
    nzind = find(abs.(D * out_genlasso[:theta]) .> 1e-8);
    fdr[2,i]      = 1 - length(findin(tind, nzind))/length(nzind);
    power[2,i]    = length(findin(tind, nzind))/length(tind)
    nzind = find(abs.(D * out_l0pen[:theta]) .> 1e-8);
    fdr[3,i]      = 1 - length(findin(tind, nzind))/length(nzind);
    power[3,i]    = length(findin(tind, nzind))/length(tind)
        
    mse_bgsm[i] = norm(theta - out_bgsm[:theta])^2/n;
    mse_genlasso[i] = norm(theta - out_genlasso[:theta])^2/n;
    mse_l0pen[i] = norm(theta - out_l0pen[:theta])^2/n;
    
    theta_bgsm[:,i] = out_bgsm[:theta];
    theta_genlasso[:,i] = out_genlasso[:theta];
    theta_l0pen[:,i] = out_l0pen[:theta];
end
fit1[:,2] = copy(mse_bgsm);
fit2[:,2] = copy(mse_genlasso);
fit3[:,2] = copy(mse_l0pen);
fitfdr[:,2] = mean(fdr, 2)[:];
fitpower[:,2] = mean(power, 2)[:];
fitt[:,2] = mean(t, 2)[:];




srand(2018);
for i = 1:10
    y     = theta + .1 * randn(1000);
    tic();
    out_genlasso = genlasso(y);
    t[2,i] = toq();
    print("genlasso done; ")
    tic();
    out_bgsm     = BGSM(y, D, mult = 1.3, num = 20, st = 1e-3 * 2);
    t[1,i] = toq();
    print("bgsm done; ")
    tic();
    out_l0pen    = l0pen(y; B = 10, mult = 2.0);
    t[3,i] = toq();
    print("l0pen done; \n")
    nzind = find(abs.(D * out_bgsm[:theta]) .> 1e-8);
    fdr[1,i]      = 1 - length(findin(tind, nzind))/length(nzind);
    power[1,i]    = length(findin(tind, nzind))/length(tind)
    nzind = find(abs.(D * out_genlasso[:theta]) .> 1e-8);
    fdr[2,i]      = 1 - length(findin(tind, nzind))/length(nzind);
    power[2,i]    = length(findin(tind, nzind))/length(tind)
    nzind = find(abs.(D * out_l0pen[:theta]) .> 1e-8);
    fdr[3,i]      = 1 - length(findin(tind, nzind))/length(nzind);
    power[3,i]    = length(findin(tind, nzind))/length(tind)
        
    mse_bgsm[i] = norm(theta - out_bgsm[:theta])^2/n;
    mse_genlasso[i] = norm(theta - out_genlasso[:theta])^2/n;
    mse_l0pen[i] = norm(theta - out_l0pen[:theta])^2/n;
    
    theta_bgsm[:,i] = out_bgsm[:theta];
    theta_genlasso[:,i] = out_genlasso[:theta];
    theta_l0pen[:,i] = out_l0pen[:theta];
end
fit1[:,1] = copy(mse_bgsm);
fit2[:,1] = copy(mse_genlasso);
fit3[:,1] = copy(mse_l0pen);
fitfdr[:,1] = mean(fdr, 2)[:];
fitpower[:,1] = mean(power, 2)[:];
fitt[:,1] = mean(t, 2)[:];




genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done; l0pen done; 
genlasso done; bgsm done;